In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler
!pip install Lifetimes
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
import matplotlib.pyplot as plt
import datetime as dt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     |████████████████████████████████| 584 kB 923 kB/s eta 0:00:01
  Created wheel for autograd: filename=autograd-1.3-py3-none-any.whl size=47989 sha256=d4f534fc59e5c5d2ab95a8fa202305ca8e9c77e3bdd87aba9d8842ea5572d10b
  Stored in directory: /root/.cache/pip/wheels/ef/32/31/0e87227cd0ca1d99ad51fbe4b54c6fa02afccf7e483d045e04
Successfully built autograd
/kaggle/input/ecommerce-data/data.csv


In [2]:
#load data into pandas dataframe..
df_ = pd.read_csv("../input/ecommerce-data/data.csv", encoding="ISO-8859-1")

df=df_.copy()
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


 # 1.Data Preparation  
* # First Line:Returned Invoices were excluded in data 
* # Second Line :Negative Quantities were excluded in data 
* # Third Line:NA values were exluded in data
* # Fourth Line:Total Price was calculated with "Quatitiy x UnitPrice
* # Fifth Line:New Table was created "cltv_c" and Distinc Sum InvoiceNo , Total Quatities,Total Prices grouped by CustomerBase.
* # Sixth Line:Today date created for Frequency Value .





In [3]:
df = df[~df["InvoiceNo"].str.contains("C", na=False)]
df = df[(df['Quantity'] > 0)]
df.dropna(inplace=True)
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

cltv_c = df.groupby('CustomerID').agg({'InvoiceNo': lambda x: x.nunique(),
                                        'Quantity': lambda x: x.sum(),
                                        'TotalPrice': lambda x: x.sum()})
cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']


cltv_c.head()

,total_transaction,total_unit,total_price
CustomerID,,,
12346.0,1,74215,77183.60
12347.0,7,2458,4310.00
12348.0,4,2341,1797.24
12349.0,1,631,1757.55
12350.0,1,197,334.40


In [4]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


New Values are shown below after data cleaning and thrasholds process

Thrashold values were excluded below

In [5]:
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "UnitPrice")
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397924.0,11.833736,25.534512,1.0,2.00,6.00,12.00,298.50
UnitPrice,397924.0,2.893163,3.227059,0.0,1.25,1.95,3.75,37.06
CustomerID,397924.0,15294.315171,1713.169877,12346.0,13969.00,15159.00,16795.00,18287.00
TotalPrice,397924.0,22.394749,309.055588,0.0,4.68,11.80,19.80,168469.60


Old Values are shown below before data cleaning and thrasholds process

In [6]:
df_.describe().T


,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


If we  compare the numerical variables describe function show us data cleaning are beneficial for our modelling. 

# 2. Average Order Value (average_order_value = total_price / total_transaction)

In [7]:
cltv_c['avg_order_value'] = cltv_c['total_price'] / cltv_c['total_transaction']

# 3. Purchase Frequency (total_transaction / total_number_of_customers)

In [8]:
cltv_c["purchase_frequency"] = cltv_c['total_transaction'] / cltv_c.shape[0]


# 4. Repeat Rate & Churn Rate (amount of customers at least 2  shopping   / all customers)

In [9]:
repeat_rate = cltv_c[cltv_c.total_transaction > 1].shape[0] / cltv_c.shape[0]
churn_rate = 1 - repeat_rate

# 5. Profit Margin (profit_margin =  total_price * 0.10)

In [10]:
cltv_c['profit_margin'] = cltv_c['total_price'] * 0.10

cltv_c.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin
CustomerID,,,,,,
12346.0,1,74215,77183.60,77183.600000,0.000230,7718.360
12347.0,7,2458,4310.00,615.714286,0.001613,431.000
12348.0,4,2341,1797.24,449.310000,0.000922,179.724
12349.0,1,631,1757.55,1757.550000,0.000230,175.755
12350.0,1,197,334.40,334.400000,0.000230,33.440


# 6. Customer Value (customer_value = average_order_value * purchase_frequency)

In [11]:
cltv_c['customer_value'] = cltv_c['avg_order_value'] * cltv_c["purchase_frequency"]

# 7. Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)

In [12]:
cltv_c['cltv'] = (cltv_c['customer_value'] / churn_rate) * cltv_c['profit_margin']

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(cltv_c[["cltv"]])
cltv_c["scaled_cltv"] = scaler.transform(cltv_c[["cltv"]])


cltv_c.sort_values(by="scaled_cltv", ascending=False).head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,customer_value,cltv,scaled_cltv
CustomerID,,,,,,,,,
14646.0,74,197491,280206.02,3786.567838,0.017055,28020.602,64.578479,5.255382e+06,1.000000
18102.0,60,64124,259657.30,4327.621667,0.013828,25965.730,59.842660,4.512846e+06,0.858709
17450.0,46,69993,194550.79,4229.365000,0.010602,19455.079,44.837702,2.533468e+06,0.482071
16446.0,2,80997,168472.50,84236.250000,0.000461,16847.250,38.827495,1.899798e+06,0.361496
14911.0,201,80515,143825.06,715.547562,0.046324,14382.506,33.147052,1.384582e+06,0.263460


# 8. Creation of Segmentation


In [13]:


cltv_c["segment"] = pd.qcut(cltv_c["scaled_cltv"], 4, labels=["D", "C", "B", "A"])

cltv_c[["total_transaction", "total_unit", "total_price", "cltv", "scaled_cltv"]].sort_values(by="scaled_cltv",
                                                                                              ascending=False).head()

cltv_c.groupby("segment")[["total_transaction", "total_unit", "total_price", "cltv", "scaled_cltv"]].agg(
    {"count", "mean", "sum"})




total_transaction                  total_unit                     \
                      sum       mean count        sum         mean count   
segment                                                                    
D                    1373   1.265438  1085     140493   129.486636  1085   
C                    2035   1.875576  1085     310632   286.296774  1085   
B                    3805   3.510148  1084     703994   649.440959  1084   
A                   11323  10.435945  1085    4026577  3711.130876  1085   

         total_price                             cltv                      \
                 sum         mean count           sum          mean count   
segment                                                                     
D         194130.090   178.921742  1085  2.709108e+03      2.496874  1085   
C         503836.852   464.365762  1085  1.654171e+04     15.245815  1085   
B        1161895.311  1071.859143  1084  8.912941e+04     82.222707  1084   
A        7051545.651  6499.120416  1085  2.457456e+07  22649.366890  1085   

        scaled_cltv                      
                sum          mean count  
segment                                  
D          0.000515  4.751079e-07  1085  
C          0.003148  2.900991e-06  1085  
B          0.016960  1.564543e-05  1084  
A          4.676075  4.309747e-03  1085

You can see the Customers Segment and related values .

In [14]:
cltv_c.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,customer_value,cltv,scaled_cltv,segment
CustomerID,,,,,,,,,,
12346.0,1,74215,77183.60,77183.600000,0.000230,7718.360,17.788338,398748.869408,0.075874,A
12347.0,7,2458,4310.00,615.714286,0.001613,431.000,0.993316,1243.380187,0.000237,A
12348.0,4,2341,1797.24,449.310000,0.000922,179.724,0.414206,216.202920,0.000041,A
12349.0,1,631,1757.55,1757.550000,0.000230,175.755,0.405059,206.759170,0.000039,A
12350.0,1,197,334.40,334.400000,0.000230,33.440,0.077068,7.484830,0.000001,C
